In [1]:
import pandas as pd
df = pd.read_parquet('allcausedementia.parquet', engine = 'fastparquet')
df_demographics = pd.read_parquet('../../../randy/proj_idp/tidy_data/demographics/demographics_df.parquet', engine = 'fastparquet')

patient_df = pd.read_parquet('raw_allsnps_AD.parquet', engine = 'fastparquet')


In [3]:
patient_df.shape

(487409, 1047)

In [2]:
first_occ = df_demographics['53-0.0']
dates = pd.to_datetime(first_occ)

curr = pd.to_datetime('2022-01-01')
add_age = curr - dates

age = df_demographics['21003-0.0']
print(age.head())
years_until = add_age.dt.total_seconds() / (365.25 * 24 * 3600)
df_demographics['curr_age'] = (years_until + age).round()


0    58.0
1    43.0
2    40.0
3    59.0
4    45.0
Name: 21003-0.0, dtype: float64


In [ ]:
df_demographics.head()
df_demographics.columns 

Index(['eid', '6138-0.0', '6138-0.1', '6138-0.2', '6138-0.3', '6138-0.4',
       '6138-0.5', '6138-1.0', '6138-1.1', '6138-1.2', '6138-1.3', '6138-1.4',
       '6138-1.5', '6138-2.0', '6138-2.1', '6138-2.2', '6138-2.3', '6138-2.4',
       '6138-2.5', '6138-3.0', '6138-3.1', '6138-3.2', '6138-3.3', '6138-3.4',
       '6138-3.5', '53-0.0', '53-1.0', '53-2.0', '53-3.0', '54-0.0', '54-1.0',
       '54-2.0', '54-3.0', '21003-0.0', '21003-1.0', '21003-2.0', '21003-3.0',
       '21003-0.0_squared', '21003-1.0_squared', '21003-2.0_squared',
       '21003-3.0_squared', '31-0.0', '21000-0.0', '21000-1.0', '21000-2.0',
       '845-0.0', '845-1.0', '845-2.0', 'curr_age'],
      dtype='object')

In [7]:
df_demographics.head()

,eid,6138-0.0,6138-0.1,6138-0.2,6138-0.3,6138-0.4,6138-0.5,6138-1.0,6138-1.1,6138-1.2,...,21003-2.0_squared,21003-3.0_squared,31-0.0,21000-0.0,21000-1.0,21000-2.0,845-0.0,845-1.0,845-2.0,curr_age
0,6025251,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4761.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,74.0
1,6025268,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,59.0
2,6025273,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1001.0,NaN,NaN,NaN,NaN,NaN,56.0
3,1000054,-7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,1001.0,NaN,NaN,17.0,NaN,NaN,71.0
4,1000078,1.0,2.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,1001.0,NaN,NaN,NaN,NaN,NaN,59.0


In [4]:
df_demographics.to_parquet('demographics.parquet', engine='fastparquet')

In [9]:
# age and sex
# age: 21003-0.0
# sex: 31-0.0
cols = ['eid', 'curr_age', '31-0.0']
select_df = df_demographics[cols]
select_df.head()

,eid,curr_age,31-0.0
0,6025251,74.0,NaN
1,6025268,59.0,NaN
2,6025273,56.0,NaN
3,1000054,71.0,0.0
4,1000078,59.0,0.0


In [11]:
merged_df = pd.merge(df, select_df, on = 'eid')

In [ ]:
merged_df.to_parquet('allcausedementia.parquet', engine = 'fastparquet')

In [8]:
# 65+ only
select_df = select_df[select_df['curr_age'] >= 65]
select_df.shape


(355146, 3)

In [9]:
# select and merge
old_df = df[df['eid'].isin(select_df['eid'])]
old_demo_df = pd.merge(old_df, select_df, on = 'eid')

In [10]:
old_demo_df.head()
old_demo_df.shape

(319429, 539)

In [10]:
old_demo_df.to_parquet('65up_sex.parquet', engine = 'fastparquet')